In [1]:
import pandas as pd
import numpy as np
from joblib import dump

In [2]:
data = pd.read_excel("komentar/data_clean_from_dataset.xlsx")
komentar = data.komentar.tolist()
label = np.array(data.label.tolist())
y = label
data.head()

,Unnamed: 0,no,label,komentar
0,0,0,non spam,waduh mara ken ae mbak via ni kirim balikpapan...
1,1,1,spam,awass akun adu domba min tu org suka nyinyirin...
2,2,2,non spam,begini kalo ga biasa lipsing sekali lipsing ku...
3,3,3,non spam,nitijen mha bebar bkin mlu 2 ajh kliaan apa sd...
4,4,4,spam,maantaap banggeet kaak ssetelaah akuu pakee ge...


# Pemodelan

## Pemodelan TF-IDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(komentar)
save = vectorizer.fit(komentar)
dump(save, "model/tfidf.w")

# print(vectorizer.get_feature_names())
print(X.shape)

(4948, 14795)


## Pemodelan SVM

In [4]:
from sklearn.svm import SVC
clf = SVC()#gamma='auto'

# Pemodelan dan Validasi Model

In [5]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

start = 0.1
stop = 1
step = start
gamma_list = np.arange(start,stop+step,step)
akurasi = list()

ix = 0 
for gamma in gamma_list:
    clf.set_params(gamma=gamma)
    temp_akurasi = list()
    
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        clf.fit(X_train,y_train)
        
        y_pred = clf.predict(X_test)
        temp_akurasi.append(accuracy_score(y_test, y_pred))
        
    rerata_akurasi = sum(temp_akurasi)/len(temp_akurasi)
    akurasi.append(rerata_akurasi)
    
    ix +=1
    if ix%100 !=0:
        print(".", end="")
    else:
        print("")

        pass

ditt = {
    "gamma":gamma_list,
    "akurasi":akurasi
}

index = akurasi.index(max(akurasi))
acc = pd.DataFrame.from_dict(ditt)
acc

..........

,gamma,akurasi
0,0.1,0.927679
1,0.2,0.941617
2,0.3,0.946060
3,0.4,0.947675
4,0.5,0.948279
5,0.6,0.949087
6,0.7,0.949087
7,0.8,0.948482
8,0.9,0.947471
9,1.0,0.946664


In [6]:
print(gamma_list[index])
print(akurasi[index])

0.6
0.9490874363327674


In [8]:
print(gamma_list[index])
print(akurasi[index])

0.6
0.9508795526770909


# Menyimpan Model

In [7]:
clf.set_params(gamma=gamma_list[index])
clf.fit(X, y)

dump(clf, "model/svm.m")

['model/svm.m']